In [1]:
from RL.Env import Env
from RL.Agent import Agent
from Training import data_spilt, ReadH5d, create_data_loader
from Training import train_seg_net, test_seg_net
from Training import test_agent
from monai.transforms import *
from monai.networks.nets import DynUNet, SwinUNETR
from monai.losses import DiceFocalLoss
import torch
import random
from torch.distributions.categorical import Categorical


batch_size=6
num_epoch=1000



seg_list, rl_list, holdout_list = data_spilt('./picai_h5', 110, 100, 10)
seg_list_promise, rl_list_promise, holdout_list_promise = data_spilt('./promise_h5', 231, 180, 20)

train_list = rl_list + rl_list_promise
test_list = holdout_list + holdout_list_promise



device = 'cpu'
seg_model = SwinUNETR(
    img_size = (128, 128, 32),
    in_channels = 2,
    out_channels = 1,
    depths = (2, 2, 2, 2),
    num_heads = (3, 6, 12, 24),
    drop_rate = 0.1,
    attn_drop_rate = 0.1,
    dropout_path_rate = 0.1,
    downsample="mergingv2",
    use_v2=True,
)
seg_model.load_state_dict(torch.load("./models/segmentation.ptm", map_location=device, weights_only=True))
seg_model.eval()



agent = Agent(gamma = 0.8, batch_size=batch_size, n_epochs=10, device=device)
agent.load_models("/home/xiangcen/RLModality/models/rl_models/actor49.ptm")
test_result_list = []


test_list = holdout_list


train_dir = random.choice(holdout_list)
agent.memory.clear_memory()
env = Env(train_dir, seg_model)
obs = env.reset()
for _ in range(10):
    action, features = agent.choose_action(obs.unsqueeze(0).to(device), noise=0.)
    print(features)
    print(action)
    next_obs, reward = env.step_train(action)
    test_result_list.append(env.calculate_current_accuracy())
    agent.remember(obs, action, reward)
    obs = next_obs


/home/xiangcen/miniconda3/envs/dl/lib/python3.12/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)


actor loaded!
tensor([1.2046e-08, 6.4558e-04, 3.3488e-03, 9.7845e-08, 9.3685e-09, 6.2621e-08,
        4.8055e-10, 8.0121e-10, 5.7596e-16, 6.9671e-06, 4.9681e-04, 1.4622e-08,
        1.8441e-11, 1.1481e-09, 4.5782e-11, 8.0133e-14, 1.4246e-06, 8.6800e-05,
        9.9528e-01, 9.2889e-08, 1.3436e-04, 7.8110e-10, 7.5624e-09, 6.9362e-11,
        1.6463e-10, 8.3785e-10, 2.4075e-11, 7.2844e-09, 9.7074e-09, 3.5547e-12,
        2.1006e-10, 8.0238e-13])
18
tensor([9.5143e-05, 1.3686e-02, 1.0493e-04, 2.7469e-02, 9.2569e-04, 1.0890e-04,
        3.2462e-05, 1.2140e-05, 3.2321e-07, 3.5471e-02, 1.6069e-04, 1.4121e-04,
        7.2087e-05, 7.2371e-04, 2.0053e-05, 3.4696e-07, 4.0538e-05, 2.6793e-05,
        6.3643e-05, 9.1946e-01, 2.2373e-04, 7.5055e-04, 8.5987e-05, 3.3323e-06,
        3.1557e-05, 1.7423e-05, 2.4071e-05, 9.4566e-07, 1.6755e-06, 2.2926e-04,
        1.2082e-05, 2.2368e-06])
19
tensor([8.7258e-04, 8.4629e-01, 5.4370e-04, 6.6748e-04, 2.3683e-03, 8.2831e-05,
        7.8228e-05, 6.6073e-05, 1.

In [2]:
state_arr, action_arr, reward_arr, batches = \
            agent.memory.generate_batches()


# feature = agent.actor(state_arr.to(device))
# dist = Categorical(torch.softmax(feature, dim=-1))
# new_probs = dist.log_prob(action_arr.to(device).flatten())
cdr_arr = agent.calculate_cdr(reward_arr)


print(action_arr)
print(reward_arr)
print(test_result_list)
# print(dice_acc)



tensor([18, 19,  1, 20,  5, 20,  6,  6, 14,  6])
tensor([1., 1., 1., 1., 1., 0., 0., 0., 1., 0.])
[0.38058832463215564, 0.6071317314156065, 0.7002528615688951, 0.7646526483761041, 0.8045744114428186, 0.8045744114428186, 0.8037187601707796, 0.8037187601707796, 0.8045744114428186, 0.8037187601707796]


In [ ]:
import matplotlib.pyplot as plt
rows = 4
cols = 8

# Create a figure with subplots
fig, axes = plt.subplots(rows, cols, figsize=(16, 8))  # Adjust figsize as needed


index = 3

# Plot each image
for i, ax in enumerate(axes.flat):
    
    ax.imshow(state_arr[index][2, :, :, i], cmap='gray')  # Use appropriate colormap (e.g., 'gray' or 'viridis'), cmap='gray'
    # ax.imshow(env.both_seg[0, :, :, i], cmap='gray')
    
    ax.axis('off')  # Hide axis


# Adjust layout
plt.tight_layout()

state_arr.max(), state_arr.min()